# Проект SQL

## Описание проекта

Компания-заказчик решила быть на волне и купила крупный сервис для чтения книг по подписке. 

**Основная цель** — проанализировать базу данных, в которой — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Задачи**:
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Описание данных**

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Формирование подключения к базе данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {
    ### private_info
}
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Обзор данных

In [2]:
query = ''' 
    SELECT *
    FROM books
    LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine).head())
query = ''' 
    SELECT *
    FROM authors
    LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine).head())
query = ''' 
    SELECT *
    FROM publishers
    LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine).head())
query = ''' 
    SELECT *
    FROM ratings
    LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine).head())
query = ''' 
    SELECT *
    FROM reviews
    LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine).head())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Анализ базы данных

**Расчёт количества книг вышедших после 1 января 2000 года**

In [3]:
query = ''' 
    SELECT *
    FROM books
    WHERE PUBLICATION_DATE >= DATE'2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
816,994,408,World War Z: An Oral History of the Zombie War,342,2006-09-12,68
817,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
818,998,201,Year of Wonders,358,2002-04-30,212
819,999,94,You Suck (A Love Story #2),328,2007-01-16,331


**Вывод**: после 1 января 2000 года вышла 821 книга. Не так уж много. Такая маленькая база новых книг может не заинтересовать потенциальных клиентов.

**Расчёт количества обзоров и средней оценки для каждой книги**

In [4]:
query = ''' 
    SELECT
        b.book_id,
        b.title,
        COUNT(distinct rev.review_id) AS review_count,
        AVG(rat.rating) AS avg_rating
    FROM books b
    LEFT JOIN reviews rev
        ON b.book_id = rev.book_id
    LEFT JOIN ratings rat
        ON b.book_id = rat.book_id
    GROUP BY b.book_id, b.title
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,review_count,avg_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


Группировка id+title используется для того, чтобы не схлопнуть книги с одинаковыми названиями, но при этом сами названия было видно.

**Вывод**: в целом, видно, что есть книги как популярные, так и малоизвестные, как с высокими рейтингами, так и с низкими. Также можно заметить, что активность читателей присутствует (т.е. люди пишут отзывы и ставят оценки).

**Определение издательства, которое выпустило наибольшее число книг толще 50 страниц (исключаются брошюры)**

In [8]:
query = ''' 
    SELECT 
        p.publisher,
        COUNT(b.book_id)
    FROM books b
    LEFT JOIN publishers p
        ON b.publisher_id = p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher
    ORDER BY 2 DESC
    LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine) 

,publisher,count
0,Penguin Books,42


**Вывод**: самое популярное издательство среди книг, представленных в базе - Penguin Books. Издательство выпустило 42 книги.

**Определение автора с самой высокой средней оценкой книг (учитываются только книги с 50 и более оценками)**

In [11]:
query = ''' 
    WITH t1 AS(
        SELECT 
            book_id
        FROM ratings
        GROUP BY book_id
        HAVING COUNT(rating) >= 50
        )
        
    SELECT a.author, AVG(r.rating)
    FROM books b
    LEFT JOIN authors a
        ON b.author_id = a.author_id
    INNER JOIN t1
        ON t1.book_id = b.book_id
    LEFT JOIN ratings r
        ON b.book_id = r.book_id
    GROUP BY a.author
    ORDER BY 2 DESC
    LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine) 

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


**Вывод**: установлен самый высокооцениваемый автор. Стоит загрузить побольше книг данного автора.

**Расчёт среднего количества обзоров от пользователей, которые поставили больше 50 оценок**

In [12]:
query = ''' 
    WITH t1 AS(
        SELECT 
            username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating) > 50
    ),
    t2 AS (
        SELECT 
            username,
            COUNT(book_id) AS count_reviews
        FROM reviews
        GROUP BY username
    )
    SELECT
        AVG(count_reviews) AS avg_reviews
    FROM t2
    WHERE t2.username IN (SELECT username FROM t1) --или можно сделать через джоин и условие на null
'''
pd.io.sql.read_sql(query, con = engine) 

,avg_reviews
0,24.333333


**Вывод**: в среднем пользователи, которые ставят много оценок пишут более 24 обзоров. Нужно стимулировать таких пользователей для увеличения привлекательности сервиса.